In [ ]:
import cv2
import os
import time
from memory_queues import square_queue, gui_queue
from memory_logic import register_card
from sift_utils import extract_sift_signature, auto_crop_inside_white_edges, draw_oriented_bounding_box
from recorded_positions import pick_positions, drop_positions, home_pose, scan_pose
from pyniryo2 import NiryoRos, NiryoRobot, Vision
import pyniryo
from matplotlib import pyplot as plt
import numpy as np

In [ ]:
robot_ip_address = "169.254.200.200"
robot = NiryoRobot(robot_ip_address)
robot.arm.calibrate_auto()
# robot.activate_vacuum()
robot.tool.release_with_tool()
robot.tool.grasp_with_tool()
robot.arm.move_pose(home_pose)


In [19]:
pose_read = robot.arm.get_pose()
print(f'{round(pose_read.x,2), round(pose_read.y,2), round(pose_read.z,2), round(pose_read.roll,2), round(pose_read.pitch,2), round(pose_read.yaw)} ')

(0.45, 0.04, 0.25, -1.54, -0.02, 1) 


In [ ]:
#L1
robot.arm.move_pose([0.45, 0.01, 0.2, -0.54, -0.98, 0.5])

In [8]:
#home pose 
robot.arm.move_pose([0.33, 0.01, 0.19, 0.25, 1.45, 0.314])

In [ ]:
#scan pose
robot.arm.move_pose([0.45, 0.04, 0.25, -1.54, -0.02, 1])

In [6]:
robot.arm.move_pose([0.38, 0.15, 0.05, 0.87, 1.51, 0.9])

In [9]:
#scan pose
robot.arm.move_pose([0.45, -0.02, 0.19, 0.04, -1.07, 0])


In [15]:
robot.tool.grasp_with_tool()

In [12]:
robot.tool.release_with_tool()


In [ ]:
robot.tool.release_with_tool()
robot.tool.grasp_with_tool()
robot.arm.move_pose([-0.02, -0.46, 0.18, -0.13, -1.15, -1.504])


In [ ]:
from pyniryo2 import NiryoRobot
import time

# Define home position
home_pose = [0.26, -0.01, 0.22, 2.22, 1.48, 2]

# All drop positions from A1 to D5
drop_positions = {
    "A1": [0.39, 0.13, 0.07, 2.05, 1.47, 2.29],
    "A2": [0.39, 0.07, 0.07, 2.63, 1.43, 2.74],
    "A3": [0.39, 0.02, 0.07, 2.88, 1.44, 2.9],
    "A4": [0.38, -0.03, 0.07, -3.05, 1.47, -3.13],
    "A5": [0.38, -0.08, 0.07, -2.49, 1.47, -2.65],
    "B1": [0.33, 0.13, 0.07, 2.23, 1.44, 2.42],
    "B2": [0.32, 0.08, 0.07, 2.46, 1.47, 2.59],
    "B3": [0.32, 0.03, 0.05, 3.1, 1.46, -3.1],
    "B4": [0.32, -0.02, 0.07, -2.82, 1.44, -2.81],
    "B5": [0.31, -0.08, 0.05, -2.52, 1.43, -2.6],
    "C1": [0.27, 0.13, 0.05, 1.99, 1.48, 2.22],
    "C2": [0.27, 0.08, 0.05, 2.71, 1.47, 2.89],
    "C3": [0.26, 0.03, 0.05, 3.1, 1.48, -3.07],
    "C4": [0.26, -0.02, 0.05, -2.9, 1.46, -2.86],
    "C5": [0.25, -0.07, 0.05, -2.58, 1.44, -2.59],
    "D1": [0.21, 0.13, 0.05, 1.94, 1.47, 2.05],
    "D2": [0.21, 0.08, 0.05, 2.6, 1.47, 2.65],
    "D3": [0.21, 0.04, 0.05, 3.02, 1.45, 3.07],
    "D4": [0.2, -0.01, 0.05, -3.1, 1.41, -3],
    "D5": [0.19, -0.07, 0.05, -3.12, 1.42, 3],
}

# Connect to Niryo robot
robot = NiryoRobot("10.10.10.10")  # Update IP if needed

# Initial setup
robot.arm.calibrate_auto()
robot.tool.update_tool()
robot.tool.release_with_tool()

# Loop over all positions
for label in sorted(drop_positions.keys()):
    print(f"\n[PROCESS] Moving to drop: {label}")
    
    # Move to home
    robot.arm.move_pose(home_pose)
    
    # Activate suction to simulate pick-up
    robot.tool.grasp_with_tool()
    time.sleep(1.2)

    # Move to drop position
    robot.arm.move_pose(drop_positions[label])
    
    # Release suction to drop card
    robot.tool.release_with_tool()
    time.sleep(0.8)

    # Return to home
    robot.arm.move_pose(home_pose)

# Disconnect safely
robot.close_connection()


In [18]:

import cv2
CARD_BOX = (220, 100, 270, 270)  

ros_instance = NiryoRos("169.254.200.200")
vision = Vision(ros_instance)

print(f"[SCAN] Looking for card at ")
last_center = stable_since = detection_time = None
while True:
        
    img_compressed = vision.get_img_compressed()
    img_uncompressed = pyniryo.uncompress_image(img_compressed)
    camera_info = vision.get_camera_intrinsics()
    img = pyniryo.undistort_image(img_uncompressed, camera_info.intrinsics, camera_info.distortion)

    frame = cv2.resize(img, (640, 480))
    cv2.rectangle(frame, (CARD_BOX[0], CARD_BOX[1]), (CARD_BOX[0]+CARD_BOX[2], CARD_BOX[1]+CARD_BOX[3]), (0, 255, 0), 2)
    cv2.imshow("Adjust ROI", frame)

    key = cv2.waitKey(1)
    if key == ord('q'): break


cv2.destroyAllWindows()

[SCAN] Looking for card at 


KeyboardInterrupt: 